# 初始化 摄像头 调用函数

In [1]:
import cv2
 
class VideoCamera(object):
    def __init__(self):
        # 定义视频数据的图像来源
        self.video = cv2.VideoCapture(0)
    
    def __del__(self):
        self.video.release()
    
    # 设置获取视频的图像数据
    def get_frame(self):
        # 每次运行都通过来源进行数据获取
        success, image = self.video.read()
        # 将图片进行编码，转化为指定的图片格式
        ret, jpeg = cv2.imencode('.jpg', image)
        # 通过字节形式进行流传输
        return jpeg.tobytes()

In [ ]:
import os
from flask import Flask, render_template, Response, make_response
app = Flask(__name__)
 
# 相机推流 - 创建对象 - 根据图生成二进制传输流
def gen(camera):
    while True:
        frame = camera.get_frame()
        # 生成二进制传输流 同时生成包含头部数据和图片数据的传输流
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

# 每次访问，即返回一张处理好的照片 - 注意此处的response是用来进行数据的 反馈操作
@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
 
#当前实时相机画面
@app.route('/cur_camera')
def cur_camera():
    return render_template(r'Flask-StreamData-Trans.html')
 
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.241.8:5000/ (Press CTRL+C to quit)
192.168.241.151 - - [04/Oct/2022 15:47:51] "GET / HTTP/1.1" 404 -
192.168.241.151 - - [04/Oct/2022 15:48:09] "GET /cur_camera HTTP/1.1" 200 -
192.168.241.151 - - [04/Oct/2022 15:48:19] "GET /video_feed HTTP/1.1" 200 -
